# stackview in combination with Voila
If you combine stackview with [voila](https://voila.readthedocs.io/en/stable/), you can build powerful, interactive, browser-based image processing apps and demonstrators. 
Just build a graphical user interface in a Jupyter notebook and then start it using
```
voila apoc_gui.ipynb
```

In [1]:
from skimage.io import imread
import numpy as np
import stackview
import ipywidgets as widgets
from IPython.display import display, clear_output
import apoc
import pyclesperanto_prototype as cle

In [2]:
image = imread('data/blobs.tif')

In [3]:
annotation = cle.push(np.zeros_like(image, dtype=np.uint32)).astype(np.uint32)
labels = cle.push(np.zeros_like(image, dtype=np.uint16)).astype(np.uint32)

In [4]:
# Buttons
train_button = widgets.Button(
                                    description='Train', 
                                    button_style='success', 
                                    tooltip='Start machine learning training')
train_button.layout.height = '500'

def on_train_click(change):
        model_filename = "object_segmenter.cl"
        apoc.erase_classifier(model_filename)
        model = apoc.ObjectSegmenter(model_filename)
        model.train(apoc.PredefinedFeatureSet.small_quick.value, image=image, ground_truth=annotation)
        new_labels = model.predict(image=image)
        cle.copy(new_labels, labels)
        slice_placeholder.update()

train_button.on_click(on_train_click)

reset_button = widgets.Button(
                                    description='Reset', 
                                    button_style='danger', 
                                    tooltip='Reset annotation')

def on_reset_click(change):
    cle.set(annotation, 0)
    cle.set(labels, 0)        
    annotate_placeholder.update()
    slice_placeholder.update()
        
reset_button.on_click(on_reset_click)

# Image interaction
annotate_placeholder = stackview.annotate(image, annotation)
slice_placeholder = stackview.curtain(image, labels, alpha=0.7)

# Initially display the annotate field
display(widgets.HBox([
    widgets.VBox([
        annotate_placeholder, 
        widgets.HBox([
            reset_button,
            train_button])
        ]),
    slice_placeholder
]))

